In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Branching

Here, we would like to create a system which:
* first, it takes the feedback (from a customer)
* second, categorize the feedback (positive/negative/neutral),
* lastly, based on the customer's feedback it answer the customer.

For this, we need to have something called branching (if statement).

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [3]:
# Load the LLM model
llm = ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0)

In [4]:
# Create prompt template
# classification prompt
classification_prompt_template = ChatPromptTemplate(
    [
        ('system', 'you are a helpful assistant.'),
        ('user', "Classify the sentiment of the following feedback as positive, negative, neutral, or escalate {feedback}")
    ]
)


positive_feedback_template = ChatPromptTemplate(
    [
        ('system', 'you are a helpful assistant.'),
        ('user', "generate a thank you message to this positive feedback {feedback}")
    ]
)
negative_feedback_template = ChatPromptTemplate(
    [
        ('system', 'you are a helpful assistant.'),
        ('user', "generate a response to this negative feedback {feedback}")
    ]
)
neutral_feedback_template = ChatPromptTemplate(
    [
        ('system', 'you are a helpful assistant.'),
        ('user', "generate a message and ask more details for this neutral feedback {feedback}")
    ]
)
escalate_feedback_template = ChatPromptTemplate(
    [
        ('system', 'you are a helpful assistant.'),
        ('user', "generate a message to escalate this feedback to a human agent {feedback}")
    ]
)

In [5]:
from langchain_core.runnables import RunnableBranch

In [6]:
# create a chain to classify the sentiment
classification = classification_prompt_template | llm | StrOutputParser()

# define a runnable branch to handle different types of feedback
branches = RunnableBranch(
    (
        lambda x: 'positive' in x,
        positive_feedback_template | llm | StrOutputParser()
    ),
    (
        lambda x: 'negative' in x,
        negative_feedback_template | llm | StrOutputParser()
    ),
    (
        lambda x: 'neutral' in x,
        neutral_feedback_template | llm | StrOutputParser()
    ),
    escalate_feedback_template | llm | StrOutputParser()
)

In [7]:
# combine classification chain with runnable branch
chain = classification | branches

In [9]:
review = "I bought a new laptop and it does not work well."
chain.invoke({'feedback': review})

"I'm sorry to hear that you are not satisfied with your experience. Your feedback is important to us, and we would like to understand more about your concerns so that we can address them effectively. Please feel free to provide more details or suggestions on how we can improve. Thank you for sharing your thoughts with us."